In [1]:

import os
os.chdir("../")
%pwd

'/home/anonymous/PycharmProjects/MLOps_heart_disease'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [3]:
from src.mlhrtds.constants import *
from src.mlhrtds.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [5]:
import os
import urllib.request as request
import zipfile
from mlhrtds import logger
from mlhrtds.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"{unzip_path} has successfully been extracted!")
            

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    logger.info(f"Data Ingestion stage completed!")
except Exception as e:
    raise e

[2023-09-18 20:11:37,203: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-18 20:11:37,207: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-18 20:11:37,210: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-18 20:11:37,211: INFO: common: created directory at: artifacts]
[2023-09-18 20:11:37,213: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-18 20:11:37,214: INFO: 310858993: File already exists of size: ~ 3 KB]
[2023-09-18 20:11:37,216: INFO: 310858993: artifacts/data_ingestion has successfully been extracted!]
[2023-09-18 20:11:37,217: INFO: 515101841: Data Ingestion stage completed!]
